# Find all must support dependent profiles in StructureDefinitions

- see FHIR-39723
- for hard coded path search all SD (in YAML) for all reference with must support = true profiles
- list of other profile that need to be supported
- incorporate into narrative generator...
- todo think about canonical too and !uscdi requirements as well! and inherited references from base. (such as Media) use snapshot instead?
  - check if inherited reference if emopty ( e.g. Media )
- associated url with title to publish  (see narrative generator)


In [3]:
from pathlib import Path
from json import load, dumps
from yaml import load as y_load, dump as y_dump, FullLoader
from fhir.resources import construct_fhir_element
from pprint import pprint
import pandas as pd
from IPython import display
yml_path = Path('/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml')
snapshot_path =  Path('/Users/ehaas/Documents/FHIR/US-Core/output')  # use this 
my_path = snapshot_path
my_path

PosixPath('/Users/ehaas/Documents/FHIR/US-Core/output')

In [4]:
mydict = {}
excludelist = []

whitelist = []

count = 0
cant_match_path = []

for enum,i in enumerate(my_path.glob('Struct*.json')):
    my_set = set()
    obj = y_load(i.read_text(),Loader=FullLoader) #dict
    if obj['id'] in excludelist or obj['type'] == 'Extension':
        continue
    # print(enum,i,)
    # print(obj['url'])
    #find references type elements in diff
    for element in obj['differential']['element']:
        # print(element['id'])
        # print()
    
        try:
            targetProfile = (type for type in element['type'] if type['code'] == 'Reference').__next__()['targetProfile']
        except KeyError:
            # find snapshot_element
            try:
                snapshot_element =  (snapshot_element for snapshot_element in obj['snapshot']['element'] if snapshot_element['id'] == element['id']).__next__()
                # check if snapshot_element has a type reference
            except StopIteration:
                # print('id mismatch')
                pass
            else:
                try:
                    snapshot_reference = (type for type in snapshot_element['type'] if type['code'] == 'Reference').__next__()['targetProfile']
                    # print(f"++++++++++++++++snapshot_reference={snapshot_reference}+++++++++++++++++++++")
                    my_set.update(snapshot_reference) #add list to set
                except KeyError:
                    # print('not a type = reference')
                    pass
                except StopIteration:
                    # print('not a type = reference')
                    pass
        except StopIteration:
            # print('no reference')
            pass
        else:
            ms_ext = None
            try:
                ms_ext = (type for type in element['type'] if type['code'] == 'Reference').__next__()['_targetProfile']
            except KeyError:
                # print('no  _targetProfile')
                pass
            except StopIteration:
                # print('stopIteration')
                pass

            # print(targetProfile, ms_ext)
            
            # print(element['id'])
            # print(dict(zip(targetProfile,ms_ext)))
            # print()
            try:
                zipped = dict(zip(targetProfile,ms_ext))
                ms_targetProfile = [k for k,v in zipped.items() if v["extension"][0]['valueBoolean']]
                # print(f"{element['id']} = {ms_targetProfile}\n")
                my_set.update(ms_targetProfile) #add list to set
            except TypeError:
                # print(f"{element['id']} = {targetProfile}\n")
                my_set.update(targetProfile) #add list to set
            
    # print(f'{my_set}\n')
    mydict[obj['url']] = list(my_set)
pprint(mydict)


{'http://hl7.org/fhir/us/core/StructureDefinition/head-occipital-frontal-circumference-percentile': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-bmi-for-age': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-weight-for-height': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-allergyintolerance': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient'],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-blood-pressure': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-bmi': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-body-height': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-body-temperature': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-body-weight': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careplan': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient'],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careteam':

AttributeError: module 'pandas' has no attribute 'DataFramet'

In [39]:
# create two lists: one for the keys and one for the values
keys = list(mydict.keys())
r_types_keys = ['Source Type' for k in keys]
values = list(mydict.values())



t_types_values = []
t_profile_values = []
for v in values:
    target_type = []
    for i,j in enumerate(v):
        target_type.append(f"Target Type {i+1}")
    # print(target_type)
    t_types_values.append('</br>'.join(target_type))
    t_profile_values.append('</br>'.join(v))




df = pd.DataFrame({'key': keys, 'Source Type':r_types_keys, 'Target Profiles': t_profile_values,  'Target Type':t_types_values})
my_md_table = df.to_markdown(index=False)
display.Markdown(my_md_table)
# display.Markdown("foo</br>bar")


| key                                                                                             | Source Type   | Target Profiles                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type                                                                                                               |
|:------------------------------------------------------------------------------------------------|:--------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------|
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab                         | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-specimen                                                                                                                                                                                                                                                                                                                                                                                                                                              | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-careteam                                | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-relatedperson</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitionerrole</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                               | Target Type 1</br>Target Type 2</br>Target Type 3</br>Target Type 4                                                       |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-pregnancystatus             | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-respiratory-rate                        | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-heart-rate                              | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-body-temperature                        | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner                            | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/pediatric-bmi-for-age                           | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/pediatric-weight-for-height                     | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                 | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-pulse-oximetry                          | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-condition-encounter-diagnosis           | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                         | Target Type 1</br>Target Type 2                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-coverage                                | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                      | Target Type 1</br>Target Type 2                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-immunization                            | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-provenance                              | Source Type   | http://hl7.org/fhir/StructureDefinition/Resource</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization                                                                                                                                                                                                                                                                                                                                                                                     | Target Type 1</br>Target Type 2                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-smokingstatus                           | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-relatedperson                           | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-sexual-orientation          | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-documentreference                       | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                | Target Type 1</br>Target Type 2</br>Target Type 3                                                                         |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-head-circumference                      | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationrequest                       | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter</br>http://hl7.org/fhir/StructureDefinition/Observation</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner</br>http://hl7.org/fhir/StructureDefinition/Condition</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-medication</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                           | Target Type 1</br>Target Type 2</br>Target Type 3</br>Target Type 4</br>Target Type 5</br>Target Type 6                   |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-specimen                                | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter                               | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-condition-problems-health-concerns</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-condition-encounter-diagnosis</br>http://hl7.org/fhir/StructureDefinition/Location</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                               | Target Type 1</br>Target Type 2</br>Target Type 3</br>Target Type 4</br>Target Type 5</br>Target Type 6                   |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-body-height                             | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-occupation                  | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-bmi                                     | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization                            | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-procedure                               | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-careplan</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-servicerequest                                                                                                                                                                                                                                                                                               | Target Type 1</br>Target Type 2</br>Target Type 3                                                                         |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-medication                              | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-screening-assessment        | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-questionnaireresponse</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-screening-assessment</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                | Target Type 1</br>Target Type 2</br>Target Type 3                                                                         |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-blood-pressure                          | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-result             | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-condition-problems-health-concerns      | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-note                   | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-result</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner</br>http://hl7.org/fhir/StructureDefinition/Media</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient | Target Type 1</br>Target Type 2</br>Target Type 3</br>Target Type 4</br>Target Type 5</br>Target Type 6</br>Target Type 7 |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-simple-observation                      | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitionerrole                        | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization</br>http://hl7.org/fhir/StructureDefinition/Endpoint</br>http://hl7.org/fhir/StructureDefinition/Location</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner</br>http://hl7.org/fhir/StructureDefinition/HealthcareService                                                                                                                                                                                         | Target Type 1</br>Target Type 2</br>Target Type 3</br>Target Type 4</br>Target Type 5                                     |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-careplan                                | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-servicerequest                          | Source Type   | http://hl7.org/fhir/StructureDefinition/DocumentReference</br>http://hl7.org/fhir/StructureDefinition/Observation</br>http://hl7.org/fhir/StructureDefinition/DiagnosticReport</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner</br>http://hl7.org/fhir/StructureDefinition/Condition</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                             | Target Type 1</br>Target Type 2</br>Target Type 3</br>Target Type 4</br>Target Type 5</br>Target Type 6                   |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-goal                                    | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-location                                | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization                                                                                                                                                                                                                                                                                                                                                                                                                                          | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-lab                    | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                          | Target Type 1</br>Target Type 2</br>Target Type 3                                                                         |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-implantable-device                      | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-allergyintolerance                      | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-questionnaireresponse                   | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                      | Target Type 1</br>Target Type 2                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/head-occipital-frontal-circumference-percentile | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-body-weight                             | Source Type   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                           |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationdispense                      | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-medication</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationrequest</br>http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                        | Target Type 1</br>Target Type 2</br>Target Type 3</br>Target Type 4</br>Target Type 5                                     |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-pregnancyintent             | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type 1                                                                                                             |
| http://hl7.org/fhir/us/core/StructureDefinition/us-core-vital-signs                             | Source Type   | http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient                                                                                                                                                                                                                                                                                                                                                                                                                                               | Target Type 1                                                                                                             |